In [1]:
import pandas as pd

In [2]:
def regression_table(params):
    
    # DECONSTRUCT PARAMS
    dataset = params['dataset']
    labels = dataset['label']
    ensemble = params['ensemble']
    
    # CALCULATE AVERAGE PREDICTION USING ENSEMBLE
    averages = mean_predictions(ensemble)
    
    # CREATE NEW DATAFRAME
    dataframe = pd.DataFrame(labels, index=dataset.index[-len(averages):])
    
    # ADD ENSEMBLE PREDICTIONS AS COLUMN
    dataframe['ensemble'] = averages
    
    # LOOP THROUGH ENSEMBLE MODELS
    for index, model in enumerate(ensemble):

        # EXTRACT NAME & PREDICTIONS
        name = model['name'] + '_' + str(index)
        data = model['result']['predictions']

        # ADD MODEL PREDICTIONS AS COLUMN
        dataframe[name] = data
        
    return dataframe

In [3]:
def mean_predictions(ensemble):
    
    # CONTAINER
    averages = []
    
    # ENSEMBLE PARAMS
    predictions = len(ensemble[0]['result']['predictions'])
    models = len(ensemble)
    
    # LOOP THROUGH EACH PREDICTION INDEX
    for index in range(predictions):
        value = 0

        # QUERY EVERY MODELS PREDICTION
        for item in ensemble:
            value += item['result']['predictions'][index]

        # CALCULATE THE MEAN VALUE & APPEND
        mean = value / models
        averages.append(mean)
        
    return averages